<a href="https://colab.research.google.com/github/ronaldofocus7/SpotifyEDA/blob/main/%E0%B9%80%E0%B8%8A%E0%B9%87%E0%B8%84_Lot_XM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Lot Avenger
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date, timedelta
import requests
import pandas as pd
import math

# ✅ API Token
API_TOKEN = "g11zgLqNRexyGXNe3n815GGK3e+fajAtn4D6UdvtE2I="
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

# ✅ Allowed Instruments
ALLOWED_INSTRUMENTS = set([
    "AUDUSD", "EURUSD", "USDCAD", "USDJPY", "AUDCAD", "AUDCHF", "AUDJPY", "AUDNZD",
    "CADCHF", "CADJPY", "CHFJPY", "EURAUD", "EURCAD", "EURCHF", "EURDKK", "EURGBP",
    "EURHKD", "EURJPY", "GBPJPY", "GBPUSD", "GOLD", "NZDCAD", "NZDUSD", "USDCHF",
    "USDHKD", "XAUEUR", "EURNZD", "GBPAUD", "GBPCAD", "GBPCHF", "NZDJPY", "USDCNH",
    "USDDKK", "USDTRY", "NZDCHF", "USDMXN", "USDSEK", "CHFSGD", "EURNOK", "EURPLN",
    "EURSEK", "EURSGD", "EURTRY", "GBPDKK", "GBPNOK", "GBPNZD", "GBPSEK", "NZDSGD",
    "SGDJPY", "SILVER", "USDHUF", "USDNOK", "USDPLN", "USDSGD", "USDZAR", "EURHUF",
    "EURZAR", "GBPSGD", "USDRUB", "EURRUB", "BTCUSD"
])

# ✅ Widgets
timeframe_dropdown = widgets.Dropdown(
    options=["เดือนนี้", "เดือนที่แล้ว", "ระบุวันที่เอง"],
    value="เดือนนี้",
    description="ช่วงเวลา:",
)

start_picker = widgets.DatePicker(description='เริ่มต้น:')
end_picker = widgets.DatePicker(description='สิ้นสุด:')
run_button = widgets.Button(description="🚀 ตรวจสอบล็อต")

output = widgets.Output()

def fetch_lots(start_date, end_date):
    all_data = []
    url = "https://mypartners.xm.com/api/trader-statistics/trades"
    params = {
        "startTime": start_date.strftime("%Y-%m-%d"),
        "endTime": end_date.strftime("%Y-%m-%d")
    }

    try:
        response = requests.get(url, headers=HEADERS, params=params)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)
        if df.empty:
            return pd.DataFrame(columns=["Date", "Active Traders", "Total Lots"])
        df["closeTime"] = pd.to_datetime(df["closeTime"])
        df["Date"] = df["closeTime"].dt.strftime("%Y-%m-%d")
        df = df[df["instrument"].isin(ALLOWED_INSTRUMENTS)]
        df = df[df["campaignName"] == "Indy Avenger Team"]
        grouped = df.groupby("Date").agg({
            "loginId": "nunique",
            "lots": "sum"
        }).reset_index()
        grouped.columns = ["Date", "Active Traders", "Total Lots"]
        return grouped
    except Exception as e:
        return f"❌ ERROR: {str(e)}"

def on_run_button_click(b):
    output.clear_output()
    with output:
        today = date.today()
        if timeframe_dropdown.value == "เดือนนี้":
            start_date = today.replace(day=1)
            end_date = today
        elif timeframe_dropdown.value == "เดือนที่แล้ว":
            first_day_this_month = today.replace(day=1)
            last_day_last_month = first_day_this_month - timedelta(days=1)
            start_date = last_day_last_month.replace(day=1)
            end_date = last_day_last_month
        else:
            start_date = start_picker.value
            end_date = end_picker.value
            if not start_date or not end_date:
                print("⚠️ กรุณาเลือกวันที่ให้ครบถ้วน")
                return
        print(f"⏳ ดึงข้อมูลระหว่าง {start_date} ถึง {end_date} ...")
        result_df = fetch_lots(start_date, end_date)
        if isinstance(result_df, str):
            print(result_df)
        else:
            # ✅ คำนวณรวม
            total_row = pd.DataFrame({
                "Date": ["รวมทั้งหมด"],
                "Active Traders": [result_df["Active Traders"].sum()],
                "Total Lots": [result_df["Total Lots"].sum()]
            })

            # ✅ รวมตารางกับผลรวม
            full_result = pd.concat([result_df, total_row], ignore_index=True)
            display(full_result)

run_button.on_click(on_run_button_click)

# ✅ Display everything
display(timeframe_dropdown, start_picker, end_picker, run_button, output)


Dropdown(description='ช่วงเวลา:', options=('เดือนนี้', 'เดือนที่แล้ว', 'ระบุวันที่เอง'), value='เดือนนี้')

DatePicker(value=None, description='เริ่มต้น:')

DatePicker(value=None, description='สิ้นสุด:')

Button(description='🚀 ตรวจสอบล็อต', style=ButtonStyle())

Output()

In [ ]:
# @title Lot Avenger (with Breakdown Option)
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date, timedelta
import requests
import pandas as pd

# ✅ API Token
API_TOKEN = "g11zgLqNRexyGXNe3n815GGK3e+fajAtn4D6UdvtE2I="
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

# ✅ Allowed Instruments
ALLOWED_INSTRUMENTS = set([
    "AUDUSD", "EURUSD", "USDCAD", "USDJPY", "AUDCAD", "AUDCHF", "AUDJPY", "AUDNZD",
    "CADCHF", "CADJPY", "CHFJPY", "EURAUD", "EURCAD", "EURCHF", "EURDKK", "EURGBP",
    "EURHKD", "EURJPY", "GBPJPY", "GBPUSD", "GOLD", "NZDCAD", "NZDUSD", "USDCHF",
    "USDHKD", "XAUEUR", "EURNZD", "GBPAUD", "GBPCAD", "GBPCHF", "NZDJPY", "USDCNH",
    "USDDKK", "USDTRY", "NZDCHF", "USDMXN", "USDSEK", "CHFSGD", "EURNOK", "EURPLN",
    "EURSEK", "EURSGD", "EURTRY", "GBPDKK", "GBPNOK", "GBPNZD", "GBPSEK", "NZDSGD",
    "SGDJPY", "SILVER", "USDHUF", "USDNOK", "USDPLN", "USDSGD", "USDZAR", "EURHUF",
    "EURZAR", "GBPSGD", "USDRUB", "EURRUB", "BTCUSD"
])

# ✅ Widgets
timeframe_dropdown = widgets.Dropdown(
    options=["เดือนนี้", "เดือนที่แล้ว", "ระบุวันที่เอง"],
    value="เดือนนี้",
    description="ช่วงเวลา:",
)

breakdown_dropdown = widgets.Dropdown(
    options=["รายวัน", "รายเดือน"],
    value="รายวัน",
    description="แสดงผล:",
)

start_picker = widgets.DatePicker(description='เริ่มต้น:')
end_picker = widgets.DatePicker(description='สิ้นสุด:')
run_button = widgets.Button(description="\U0001F680 ตรวจสอบล็อต")

output = widgets.Output()

def fetch_lots(start_date, end_date, breakdown_type="รายวัน"):
    url = "https://mypartners.xm.com/api/trader-statistics/trades"
    params = {
        "startTime": start_date.strftime("%Y-%m-%d"),
        "endTime": end_date.strftime("%Y-%m-%d")
    }
    try:
        response = requests.get(url, headers=HEADERS, params=params)
        response.raise_for_status()
        df = pd.DataFrame(response.json())
        if df.empty:
            return pd.DataFrame(columns=["Period", "Active Traders", "Total Lots"])

        df["closeTime"] = pd.to_datetime(df["closeTime"])
        df = df[df["instrument"].isin(ALLOWED_INSTRUMENTS)]
        df = df[df["campaignName"] == "Indy Avenger Team"]

        if breakdown_type == "รายเดือน":
            df["Period"] = df["closeTime"].dt.to_period("M").astype(str)
        else:
            df["Period"] = df["closeTime"].dt.strftime("%Y-%m-%d")

        grouped = df.groupby("Period").agg({
            "loginId": "nunique",
            "lots": "sum"
        }).reset_index()

        grouped.columns = ["Period", "Active Traders", "Total Lots"]

        # ✅ Add Total Row
        total_row = pd.DataFrame({
            "Period": ["รวม"],
            "Active Traders": [grouped["Active Traders"].sum()],
            "Total Lots": [grouped["Total Lots"].sum()]
        })

        return pd.concat([grouped, total_row], ignore_index=True)
    except Exception as e:
        return f"\u274C ERROR: {str(e)}"

def on_run_button_click(b):
    output.clear_output()
    with output:
        today = date.today()
        if timeframe_dropdown.value == "เดือนนี้":
            start_date = today.replace(day=1)
            end_date = today
        elif timeframe_dropdown.value == "เดือนที่แล้ว":
            first_day_this_month = today.replace(day=1)
            last_day_last_month = first_day_this_month - timedelta(days=1)
            start_date = last_day_last_month.replace(day=1)
            end_date = last_day_last_month
        else:
            start_date = start_picker.value
            end_date = end_picker.value
            if not start_date or not end_date:
                print("⚠️ กรุณาเลือกวันที่ให้ครบถ้วน")
                return
        print(f"⏳ ดึงข้อมูลระหว่าง {start_date} ถึง {end_date} แบบ {breakdown_dropdown.value}...")
        result_df = fetch_lots(start_date, end_date, breakdown_dropdown.value)
        if isinstance(result_df, str):
            print(result_df)
        else:
            display(result_df)

run_button.on_click(on_run_button_click)

# ✅ Display Widgets
display(timeframe_dropdown, start_picker, end_picker, breakdown_dropdown, run_button, output)


Dropdown(description='ช่วงเวลา:', options=('เดือนนี้', 'เดือนที่แล้ว', 'ระบุวันที่เอง'), value='เดือนนี้')

DatePicker(value=None, description='เริ่มต้น:')

DatePicker(value=None, description='สิ้นสุด:')

Dropdown(description='แสดงผล:', options=('รายวัน', 'รายเดือน'), value='รายวัน')

Button(description='🚀 ตรวจสอบล็อต', style=ButtonStyle())

Output()

In [ ]:
!pip install voila

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.7 requires jupyter-client<8,>=5.3.4, but you